In [ ]:
import os
import pandas as pd
from snowflake.snowpark import Session
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Snowflake connection configuration
snowflake_params = {
    "account": os.getenv("SNOWFLAKE_ACCOUNT"),
    "user": os.getenv("SNOWFLAKE_USER"),
    "password": os.getenv("SNOWFLAKE_PASSWORD"),
    "role": os.getenv("SNOWFLAKE_ROLE"),
    "warehouse": os.getenv("SNOWFLAKE_WAREHOUSE"),
    "database": os.getenv("SNOWFLAKE_DATABASE"),
    "schema": "ERD_SCHEMA_CLEANED"
}
final_schema = os.getenv("SNOWFLAKE_STAR_SCHEMA")

# Initialize session
session = Session.builder.configs(snowflake_params).create()

def set_schema(schema):
    session.sql(f'USE SCHEMA "{snowflake_params["database"]}"."{schema}"').collect()

# Load source tables
print("🔄 Loading source tables...")
df_customer = session.table("CUSTOMER").to_pandas()
df_employee = session.table("EMPLOYEE").to_pandas()
df_artist = session.table("ARTIST").to_pandas()
df_album = session.table("ALBUM").to_pandas()
df_invoice = session.table("INVOICE").to_pandas()
df_invoiceline = session.table("INVOICELINE").to_pandas()
df_track = session.table("TRACK").to_pandas()
df_playlisttrack = session.table("PLAYLISTTRACK").to_pandas()
df_genre = session.table("GENRE").to_pandas()
df_mediatype = session.table("MEDIATYPE").to_pandas()
df_playlist = session.table("PLAYLIST").to_pandas()

# Ensure that columns exist and are correctly typed
def ensure_int_key(df, col):
    """Ensure that a column exists and is converted to an integer, handling errors."""
    if col in df.columns:
        try:
            df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0).astype(int)
        except Exception as e:
            print(f"Error while converting {col} to int: {e}")
    else:
        print(f"Column {col} is missing in the DataFrame.")
    return df

# Dimension creation functions
def create_dim_date(df_invoice):
    df_date = df_invoice[['INVOICEDATE']].dropna().drop_duplicates().reset_index(drop=True)
    df_date['DATE'] = pd.to_datetime(df_date['INVOICEDATE']).dt.date
    df_date['DAY'] = pd.to_datetime(df_date['DATE']).dt.day
    df_date['WEEK_DAY'] = pd.to_datetime(df_date['DATE']).dt.strftime('%A')
    df_date['MONTH_NAME'] = pd.to_datetime(df_date['DATE']).dt.strftime('%B')
    df_date['MONTH_NUMBER'] = pd.to_datetime(df_date['DATE']).dt.month
    df_date['QUARTER'] = pd.to_datetime(df_date['DATE']).dt.quarter
    df_date['YEAR'] = pd.to_datetime(df_date['DATE']).dt.year
    df_date['IS_HOLIDAY'] = False  # Set holiday logic if necessary
    df_date['IS_WEEKEND'] = df_date['WEEK_DAY'].isin(['Saturday', 'Sunday'])
    df_date['DATE_ID'] = range(1, len(df_date) + 1)  # Create surrogate key
    return df_date[['DATE_ID', 'DATE', 'DAY', 'WEEK_DAY', 'IS_HOLIDAY', 'IS_WEEKEND', 'MONTH_NAME', 'MONTH_NUMBER', 'QUARTER', 'YEAR']]

def create_dim_location(df_customer):
    df_location = df_customer[['CITY', 'STATE', 'COUNTRY', 'POSTALCODE']].dropna().drop_duplicates().reset_index(drop=True)
    df_location['LOCATION_ID'] = range(1, len(df_location) + 1)  # Create surrogate key
    return df_location[['LOCATION_ID', 'CITY', 'STATE', 'COUNTRY', 'POSTALCODE']]

def create_dim_album_artist(df_album, df_artist):
    # Check if 'ARTISTID' exists in df_album
    if 'ARTISTID' not in df_album.columns:
        print(" 'ARTISTID' column is missing in df_album. Checking the available columns...")
        print(df_album.columns)  # Output the column names for inspection
        # Attempt to merge with df_track if ARTISTID is in df_track
        if 'ARTISTID' in df_track.columns:
            print("'ARTISTID' found in df_track, merging with df_track...")
            df_album = df_album.merge(df_track[['ALBUMID', 'ARTISTID']], on='ALBUMID', how='left')
        else:
            print("'ARTISTID' column not found in either df_album or df_track.")
            return None

    # Proceed with merging df_album and df_artist if 'ARTISTID' is found
    df_album_artist = df_album[['ALBUMID', 'TITLE', 'ARTISTID']].dropna().drop_duplicates().reset_index(drop=True)
    
    # Ensure 'ARTISTID' exists in df_album (and in df_artist as well)
    if 'ARTISTID' in df_album.columns and 'ARTISTID' in df_artist.columns:
        df_album_artist = df_album_artist.merge(df_artist[['ARTISTID', 'NAME']], left_on='ARTISTID', right_on='ARTISTID', how='left')
    else:
        print("'ARTISTID' column not found in one of the DataFrames. Cannot merge.")

    # Create surrogate key for the dimension
    df_album_artist['ALBUM_ARTIST_ID'] = range(1, len(df_album_artist) + 1)
    
    # Return the cleaned and merged DataFrame
    return df_album_artist[['ALBUM_ARTIST_ID', 'ALBUMID', 'TITLE', 'ARTISTID', 'NAME']]

def create_dim_track(df_track, df_genre, df_mediatype):
    # Merge df_track with df_genre
    if 'GENREID' in df_track.columns and 'GENREID' in df_genre.columns:
        df_track = df_track.merge(df_genre[['GENREID', 'NAME']], left_on='GENREID', right_on='GENREID', how='left')
        df_track.rename(columns={'NAME': 'GENRE_NAME'}, inplace=True)  # Rename 'NAME' to 'GENRE_NAME'
    else:
        print(" 'GENREID' column missing in either df_track or df_genre. Cannot merge.")

    # Merge df_track with df_mediatype
    if 'MEDIATYPEID' in df_track.columns and 'MEDIATYPEID' in df_mediatype.columns:
        df_track = df_track.merge(df_mediatype[['MEDIATYPEID', 'NAME']], left_on='MEDIATYPEID', right_on='MEDIATYPEID', how='left')
        df_track.rename(columns={'NAME': 'MEDIA_TYPE_NAME'}, inplace=True)  # Rename 'NAME' to 'MEDIA_TYPE_NAME'
    else:
        print("'MEDIATYPEID' column missing in either df_track or df_mediatype. Cannot merge.")

    # Create surrogate key
    df_track['TRACK_ID'] = range(1, len(df_track) + 1)  # Surrogate key

    # Ensure that the selected columns exist before returning
    required_columns = ['TRACK_ID', 'TRACKID', 'NAME', 'ALBUMID', 'MEDIATYPEID', 'GENREID', 'GENRE_NAME', 'MEDIA_TYPE_NAME']
    
    missing_columns = [col for col in required_columns if col not in df_track.columns]
    if missing_columns:
        print(f"Missing columns: {missing_columns}")

    # Return the cleaned and merged DataFrame with only the required columns
    return df_track[['TRACK_ID', 'TRACKID', 'NAME', 'ALBUMID', 'MEDIATYPEID', 'GENREID', 'GENRE_NAME', 'MEDIA_TYPE_NAME']]

def create_dim_playlist_track(df_playlisttrack, df_playlist):
    df_playlist_track = df_playlisttrack[['PLAYLISTID', 'TRACKID']].dropna().drop_duplicates().reset_index(drop=True)
    df_playlist_track = df_playlist_track.merge(df_playlist[['PLAYLISTID', 'NAME']], left_on='PLAYLISTID', right_on='PLAYLISTID', how='left')
    df_playlist_track['PLAYLIST_TRACK_ID'] = range(1, len(df_playlist_track) + 1)  # Create surrogate key
    return df_playlist_track[['PLAYLIST_TRACK_ID', 'PLAYLISTID', 'TRACKID', 'NAME']]

def create_dim_employee(df_employee):
    df_employee = df_employee[['EMPLOYEEID', 'LASTNAME', 'FIRSTNAME', 'TITLE', 'REPORTSTO', 'BIRTHDATE', 'HIREDATE', 'ADDRESS', 'PHONE', 'FAX', 'EMAIL']].dropna().drop_duplicates().reset_index(drop=True)
    df_employee['EMPLOYEE_ID'] = range(1, len(df_employee) + 1)  # Create surrogate key
    return df_employee[['EMPLOYEE_ID', 'EMPLOYEEID', 'LASTNAME', 'FIRSTNAME', 'TITLE', 'REPORTSTO', 'BIRTHDATE', 'HIREDATE', 'ADDRESS', 'PHONE', 'FAX', 'EMAIL']]
# Dimension creation function for DimDate without WEEK_DAY
def create_dim_date(df_invoice):
    # Extract unique invoice dates
    df_date = df_invoice[['INVOICEDATE']].dropna().drop_duplicates().reset_index(drop=True)
    
    # Convert INVOICEDATE to datetime format
    df_date['DATE'] = pd.to_datetime(df_date['INVOICEDATE']).dt.date
    df_date['DAY'] = pd.to_datetime(df_date['DATE']).dt.day
    df_date['MONTH_NAME'] = pd.to_datetime(df_date['DATE']).dt.strftime('%B')
    df_date['MONTH_NUMBER'] = pd.to_datetime(df_date['DATE']).dt.month
    df_date['QUARTER'] = pd.to_datetime(df_date['DATE']).dt.quarter
    df_date['YEAR'] = pd.to_datetime(df_date['DATE']).dt.year
    df_date['IS_HOLIDAY'] = False  # Set to False, modify logic for actual holidays
    df_date['IS_WEEKEND'] = df_date['MONTH_NAME'].isin(['Saturday', 'Sunday'])
    
    # Surrogate key: DATE_ID
    df_date['DATE_ID'] = range(1, len(df_date) + 1)
    
    # Ensure all column names are in uppercase for consistency with Snowflake
    df_date.columns = df_date.columns.str.upper()

    # Return the relevant columns without WEEK_DAY
    return df_date[['DATE_ID', 'DATE', 'DAY', 'IS_HOLIDAY', 'IS_WEEKEND', 'MONTH_NAME', 'MONTH_NUMBER', 'QUARTER', 'YEAR']]

# Dimension creation function for DimInvoice
def create_dim_invoice(df_invoice):
    # Select the necessary columns and remove duplicates
    df_invoice = df_invoice[['INVOICEID', 'CUSTOMERID', 'INVOICEDATE', 'TOTAL']].dropna().drop_duplicates().reset_index(drop=True)
    
    # Convert INVOICEDATE to datetime format
    df_invoice['INVOICEDATE'] = pd.to_datetime(df_invoice['INVOICEDATE']).dt.date
    
    # Surrogate key for DimInvoice
    df_invoice['INVOICE_ID'] = range(1, len(df_invoice) + 1)
    
    # Ensure all column names are in uppercase for consistency with Snowflake
    df_invoice.columns = df_invoice.columns.str.upper()
    
    # Return the relevant columns for DimInvoice
    return df_invoice[['INVOICE_ID', 'INVOICEID', 'CUSTOMERID', 'INVOICEDATE', 'TOTAL']]


def create_dim_customer(df_customer):
    df_customer = df_customer[['CUSTOMERID', 'FIRSTNAME', 'LASTNAME', 'COMPANY', 'ADDRESS', 'PHONE', 'FAX', 'EMAIL', 'SUPPORTREPID']].dropna().drop_duplicates().reset_index(drop=True)
    df_customer['CUSTOMER_ID'] = range(1, len(df_customer) + 1)  # Create surrogate key
    return df_customer[['CUSTOMER_ID', 'CUSTOMERID', 'FIRSTNAME', 'LASTNAME', 'COMPANY', 'ADDRESS', 'PHONE', 'FAX', 'EMAIL', 'SUPPORTREPID']]

# Fact table creation logic
def create_fact_sales(df_invoiceline, df_invoice, df_track, df_album, df_artist,
                      df_customer, df_employee, df_playlisttrack, dim_location):
    df_invoiceline.columns = df_invoiceline.columns.str.upper()
    df_invoice.columns = df_invoice.columns.str.upper()
    df_track.columns = df_track.columns.str.upper()
    df_album.columns = df_album.columns.str.upper()
    df_artist.columns = df_artist.columns.str.upper()
    df_customer.columns = df_customer.columns.str.upper()
    df_employee.columns = df_employee.columns.str.upper()

    df_customer.rename(columns={
        'BILLINGCITY': 'CITY',
        'BILLINGSTATE': 'STATE',
        'BILLINGCOUNTRY': 'COUNTRY',
        'BILLINGPOSTALCODE': 'POSTALCODE'
    }, inplace=True)

    df_invoice.rename(columns={'INVOICEDATE': 'INVOICE_DATE', 'UNITPRICE': 'UNIT_PRICE'}, inplace=True)
    invoice_subset = df_invoice.drop(columns=['INVOICEDATE'], errors='ignore')
    fact = df_invoiceline.merge(invoice_subset, on='INVOICEID', how='left', suffixes=('', '_inv'))

    # Resolve duplicate 'UNITPRICE'
    if 'UNITPRICE_y' in fact.columns:
        fact['UNIT_PRICE'] = fact['UNITPRICE_y']  
        fact = fact.drop(columns=['UNITPRICE_y'])

    if 'UNITPRICE_x' in fact.columns:
        fact['UNIT_PRICE'] = fact['UNITPRICE_x']
        fact = fact.drop(columns=['UNITPRICE_x'])

    fact = fact.merge(df_customer, on='CUSTOMERID', how='left')
    if 'SUPPORTREPID' in df_customer.columns:
        fact = fact.merge(df_employee, left_on='SUPPORTREPID', right_on='EMPLOYEEID', how='left')

    fact = fact.merge(df_track, on='TRACKID', how='left')
    fact = fact.merge(df_album, on='ALBUMID', how='left')
    fact = fact.merge(df_artist, on='ARTISTID', how='left')

    location_keys = ['CITY', 'STATE', 'COUNTRY', 'POSTALCODE']
    invoice_address_keys = ['BILLINGCITY', 'BILLINGSTATE', 'BILLINGCOUNTRY', 'BILLINGPOSTALCODE']
    if all(col in fact.columns for col in location_keys):
        fact = fact.merge(dim_location, left_on=location_keys, right_on=location_keys, how='left')
    elif all(col in fact.columns for col in invoice_address_keys):
        fact = fact.merge(dim_location, left_on=invoice_address_keys, right_on=location_keys, how='left')

    if 'UNIT_PRICE' in fact.columns:
        fact['EXTENDED_PRICE'] = fact['UNIT_PRICE'] * fact['QUANTITY']

    rename_map = {
        'INVOICELINEID': 'INVOICELINE_ID',
        'INVOICEID': 'INVOICE_ID',
        'INVOICE_DATE': 'INVOICE_DATE',
        'TRACKID': 'TRACK_ID',
        'CUSTOMERID': 'CUSTOMER_ID',
        'UNIT_PRICE': 'UNIT_PRICE',
        'QUANTITY': 'QUANTITY',
        'EXTENDED_PRICE': 'EXTENDED_PRICE'
    }
    if 'SUPPORTREPID' in fact.columns:
        rename_map['SUPPORTREPID'] = 'EMPLOYEE_ID'

    fact_sales = fact.rename(columns=rename_map)

    columns_to_keep = [
        'INVOICELINE_ID', 'LOCATION_ID', 'INVOICE_ID', 'TRACK_ID', 'CUSTOMER_ID',
        'EMPLOYEE_ID', 'MEDIATYPE_ID', 'INVOICE_DATE', 'UNIT_PRICE', 'QUANTITY',
        'TOTAL_AMOUNT', 'PLAYLIST_ID', 'ALBUM_ID', 'ARTIST_ID', 'MILLISECONDS', 'BYTES'
    ]
    fact_sales = fact_sales[columns_to_keep]

    return fact_sales.reset_index(drop=True)

# Dimension creation functions called
dim_date = create_dim_date(df_invoice)
dim_location = create_dim_location(df_customer)
dim_album_artist = create_dim_album_artist(df_album, df_artist)
dim_track = create_dim_track(df_track, df_genre, df_mediatype)
dim_playlist_track = create_dim_playlist_track(df_playlisttrack, df_playlist)
dim_employee = create_dim_employee(df_employee)
dim_customer = create_dim_customer(df_customer)
dim_invoice = create_dim_invoice(df_invoice)
fact_sales = create_fact_sales(df_invoiceline, df_invoice, df_track, df_album, df_artist,
                               df_customer, df_employee, df_playlisttrack, dim_location)

# Switch to final schema
set_schema(final_schema)

print("✅ All dimensions and facts loaded into ERD_SCHEMA_STAR successfully.")

# Create the dimensions
# Load DimDate into Snowflake
try:
    session.write_pandas(dim_date, "DIM_DATE", overwrite=True)
    print("✅ DIM_DATE loaded successfully.")
except Exception as e:
    print(f"❌ Error loading DIM_DATE: {e}")

# Load DimInvoice into Snowflake
try:
    session.write_pandas(dim_invoice, "DIM_INVOICE", overwrite=True)
    print("✅ DIM_INVOICE loaded successfully.")
except Exception as e:
    print(f"❌ Error loading DIM_INVOICE: {e}")


🔄 Loading source tables...
❌ Error loading DIM_DATE: 000904 (42000): SQL compilation error: error line 1 at position 135
invalid identifier 'MONTH_NAME'
✅ DIM_INVOICE loaded successfully.
